<a href="https://colab.research.google.com/github/arthurnicolas59/BloodCellDec22---DataScientest/blob/Ludo/Blood_Cell_Model_Benchmark_images_et_masques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please change your hardware accelerator")

Default GPU Device:/device:GPU:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# **APPLICATION DU MODELE BENCHMARK SUR LES IMAGES ET MASQUES ASSOCIES**

In [ ]:
# Décompression des 17000 images
!unzip 'drive/MyDrive/Datascientest/PBC_dataset_normal_DIB.zip'

In [ ]:
# suppression d'une image présentant un défaut de format :
import os

os.remove('/content/PBC_dataset_normal_DIB/neutrophil/.DS_169665.jpg')

In [ ]:
# Décompression des 17000 masques
!unzip 'drive/MyDrive/Datascientest/Masques_predits_Unet_20072023.zip'

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: masques/neutrophil/BNE_77701_mask.png  
  inflating: masques/neutrophil/BNE_129649_mask.png  
  inflating: masques/neutrophil/BNE_279578_mask.png  
  inflating: masques/neutrophil/SNE_589271_mask.png  
  inflating: masques/neutrophil/SNE_703601_mask.png  
  inflating: masques/eosinophil/EO_992859_mask.png  
  inflating: masques/neutrophil/SNE_474619_mask.png  
  inflating: masques/eosinophil/EO_467859_mask.png  
  inflating: masques/eosinophil/EO_649430_mask.png  
  inflating: masques/neutrophil/SNE_221448_mask.png  
  inflating: masques/neutrophil/SNE_871021_mask.png  
  inflating: masques/eosinophil/EO_375973_mask.png  
  inflating: masques/neutrophil/BNE_516401_mask.png  
  inflating: masques/neutrophil/BNE_567645_mask.png  
  inflating: masques/neutrophil/BNE_951529_mask.png  
  inflating: masques/neutrophil/SNE_200666_mask.png  
  inflating: masques/neutrophil/SNE_884907_mask.png  
  inflati

In [ ]:
import glob
import pandas as pd
import cv2

# Création du dataframe présentant les chemins d'accès aux images ainsi qu'à leur masque

# Trouver tous les chemins vers les fichiers qui finissent par .png
liste_image_et_masque = glob.glob('/content/masques/*/*.png')

# # Création d'une liste comprenant les chemins d'accès aux images ainsi que le label de chaque image
liste_image_et_masque = list(map(lambda x : [x,
                                  x.split('/')[4].split('_')[0],
                                  '/content/PBC_dataset_normal_DIB/'+x.split('/')[3]+'/'+ x.split('/')[4].split('_')[0]+'_'+x.split('/')[4].split('_')[1]+'.jpg'],
                       liste_image_et_masque))

# Créer un DataFrame pandas
df_image_et_masque = pd.DataFrame(liste_image_et_masque, columns=['maskpath', 'nameLabel','filepath'])
df_image_et_masque=df_image_et_masque.replace(['SNE','BNE'],'NEUTROPHIL')
df_image_et_masque=df_image_et_masque.replace(['MY','MMY','PMY'],'IG')
df_image_et_masque['label'] = df['nameLabel'].replace(df_image_et_masque.nameLabel.unique(), [*range(len(df_image_et_masque.nameLabel.unique()))])
df_image_et_masque.sort_values('maskpath')
df_image_et_masque

,maskpath,nameLabel,filepath,label
0,/content/masques/eosinophil/EO_641461_mask.png,EO,/content/PBC_dataset_normal_DIB/eosinophil/EO_...,0
1,/content/masques/eosinophil/EO_905696_mask.png,EO,/content/PBC_dataset_normal_DIB/eosinophil/EO_...,0
2,/content/masques/eosinophil/EO_694062_mask.png,EO,/content/PBC_dataset_normal_DIB/eosinophil/EO_...,0
3,/content/masques/eosinophil/EO_112463_mask.png,EO,/content/PBC_dataset_normal_DIB/eosinophil/EO_...,0
4,/content/masques/eosinophil/EO_620959_mask.png,EO,/content/PBC_dataset_normal_DIB/eosinophil/EO_...,0
...,...,...,...,...
17087,/content/masques/lymphocyte/LY_785608_mask.png,LY,/content/PBC_dataset_normal_DIB/lymphocyte/LY_...,7
17088,/content/masques/lymphocyte/LY_32573_mask.png,LY,/content/PBC_dataset_normal_DIB/lymphocyte/LY_...,7
17089,/content/masques/lymphocyte/LY_692569_mask.png,LY,/content/PBC_dataset_normal_DIB/lymphocyte/LY_...,7
17090,/content/masques/lymphocyte/LY_877055_mask.png,LY,/content/PBC_dataset_normal_DIB/lymphocyte/LY_...,7


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paramètres du modèle
num_classes = 8
input_shape = (360, 363, 3)
batch_size = 32
height, width, color = input_shape

# Charger votre dataframe
# df_image_et_masque = pd.read_csv('votre_chemin/vers/votre_dataframe.csv')

# Créer un générateur de données pour les images et les masques
datagen = ImageDataGenerator(rescale=1./255)  # Normalisation des pixels entre 0 et 1

def image_mask_generator(df, datagen, batch_size):
    image_generator = datagen.flow_from_dataframe(
        df,
        x_col='filepath',  # Colonne contenant les chemins d'accès aux images
        y_col='nameLabel',  # Colonne contenant les labels de classe
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical',  # Pour une classification multiclasse
        shuffle=True
    )

    mask_generator = datagen.flow_from_dataframe(
        df,
        x_col='maskpath',  # Colonne contenant les chemins d'accès aux masques
        y_col='nameLabel',  # Colonne contenant les labels de classe
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical',  # Pour une classification multiclasse
        shuffle=True
    )

    while True:
        images, labels = image_generator.next()
        masks, _ = mask_generator.next()  # Les masques n'ont pas besoin de labels

        yield [images, masks], labels

# Diviser le dataframe en ensembles d'entraînement, de validation et de test
train_df, test_df = train_test_split(df_image_et_masque, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Créer les générateurs de données pour les ensembles d'entraînement, de validation et de test
train_generator = image_mask_generator(train_df, datagen, batch_size)
val_generator = image_mask_generator(val_df, datagen, batch_size)
test_generator = image_mask_generator(test_df, datagen, batch_size)

# Créer le modèle
image_input = layers.Input(shape=input_shape)
mask_input = layers.Input(shape=input_shape)
conv1 = layers.Conv2D(16, 3, padding='same', activation='relu')(image_input)
pool1 = layers.MaxPooling2D()(conv1)
conv2 = layers.Conv2D(32, 3, padding='same', activation='relu')(pool1)
pool2 = layers.MaxPooling2D()(conv2)
conv3 = layers.Conv2D(64, 3, padding='same', activation='relu')(pool2)
pool3 = layers.MaxPooling2D()(conv3)
flatten = layers.Flatten()(pool3)
dense1 = layers.Dense(128, activation='relu')(flatten)
output = layers.Dense(num_classes, activation='softmax')(dense1)

model_benchmark = models.Model(inputs=[image_input, mask_input], outputs=output)

# Compiler le modèle
model_benchmark.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle sur l'ensemble d'entraînement et valider sur l'ensemble de validation
model_benchmark.fit(train_generator, steps_per_epoch=len(train_df) // batch_size,
                    validation_data=val_generator, validation_steps=len(val_df) // batch_size,
                    epochs=10)

# Évaluer le modèle sur l'ensemble de test
test_loss, test_accuracy = model_benchmark.evaluate(test_generator, steps=len(test_df) // batch_size)
print(f"Test accuracy: {test_accuracy}")

Found 10938 validated image filenames belonging to 8 classes.
Found 10938 validated image filenames belonging to 8 classes.
Epoch 1/10
341/341 [==============================] - ETA: 0s - loss: 1.2637 - accuracy: 0.5665Found 2735 validated image filenames belonging to 8 classes.
Found 2735 validated image filenames belonging to 8 classes.
341/341 [==============================] - 114s 327ms/step - loss: 1.2637 - accuracy: 0.5665 - val_loss: 0.9798 - val_accuracy: 0.6430
Epoch 2/10
341/341 [==============================] - 113s 330ms/step - loss: 0.6408 - accuracy: 0.7767 - val_loss: 0.5218 - val_accuracy: 0.8085
Epoch 3/10
341/341 [==============================] - 111s 326ms/step - loss: 0.3544 - accuracy: 0.8750 - val_loss: 0.3940 - val_accuracy: 0.8566
Epoch 4/10
341/341 [==============================] - 111s 326ms/step - loss: 0.2388 - accuracy: 0.9171 - val_loss: 0.4004 - val_accuracy: 0.8596
Epoch 5/10
341/341 [==============================] - 111s 326ms/step - loss: 0.2127 -